In [1]:
!pip install oracledb

In [2]:
date_dim_select =  "select ORDERDATE from orders"
date_dim_insert = '''insert into date_dim(TIME_STAMP, YEAR, QUARTER, MONTH, DAY, HOUR, MINUTE)values(:1, :2, :3, :4, :5, :6, :7)'''

In [3]:
import oracledb
import pandas as pd

# Connect to the Oracle database
operational_connection = oracledb.connect(user="walmart", password="walmart", dsn="127.0.0.1:1521/freepdb1")
print("Connected to Oracle Database")
 
# Create a cursor to execute queries
operational_cursor = operational_connection.cursor()
def get_data_frame(cursor, query):
    # Execute the SQL query
    cursor.execute(query)
    # Fetch all rows from the query result
    data = cursor.fetchall()
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]
    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

Connected to Oracle Database


In [4]:
time_df = get_data_frame(operational_cursor, date_dim_select)
time_df

,ORDERDATE
0,2025-01-01 10:00:00
1,2025-01-02 14:30:00
2,2025-01-03 09:00:00
3,2025-01-04 11:45:00
4,2025-01-05 16:15:00
...,...
95,2025-04-06 12:00:00
96,2025-04-07 14:30:00
97,2025-04-08 10:45:00
98,2025-04-09 11:15:00


In [5]:
time_df['start_time'] = time_df.ORDERDATE
time_df['year'] = time_df.ORDERDATE.dt.year
time_df['quarter'] = time_df.ORDERDATE.dt.quarter
time_df['month'] = time_df.ORDERDATE.dt.month
time_df['day'] = time_df.ORDERDATE.dt.day
time_df['hour'] = time_df.ORDERDATE.dt.hour
time_df['minutes'] = time_df.ORDERDATE.dt.minute
time_df= time_df[['start_time','year','quarter','month','day','hour','minutes']]

In [6]:
time_df.head()

,start_time,year,quarter,month,day,hour,minutes
0,2025-01-01 10:00:00,2025,1,1,1,10,0
1,2025-01-02 14:30:00,2025,1,1,2,14,30
2,2025-01-03 09:00:00,2025,1,1,3,9,0
3,2025-01-04 11:45:00,2025,1,1,4,11,45
4,2025-01-05 16:15:00,2025,1,1,5,16,15


In [7]:
# Connect to the Oracle database
star_connection = oracledb.connect(user="walmart_star_schema", password="walmart", dsn="127.0.0.1:1521/freepdb1")
star_cursor = star_connection.cursor()
print("Connected to Oracle Database")
def insert_data_frame(cursor, querey, dataFrame):
    for i, row in dataFrame.iterrows():
        cursor.execute(querey, tuple(row))
    cursor.execute('commit')

Connected to Oracle Database


In [8]:
insert_data_frame(star_cursor, date_dim_insert ,time_df)